In [ ]:
!pip3 install transformers

In [4]:
# example from: https://huggingface.co/dslim/bert-base-NER.
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)

for i in ner_results:
  print(i)

{'entity': 'B-PER', 'score': 0.99901396, 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19}
{'entity': 'B-LOC', 'score': 0.999645, 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}


In [33]:
# attempt at also pulling relationship from simple sentences

from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

from nltk.tokenize import word_tokenize
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
import re

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Sarah and I live in London"

ner_results = nlp(example)

ner_words = []

# identify "start" word and "end" word
for i in ner_results:
  word = i.get('word')
  ner_words.append(word)

# find what is between start and end word
tagging_prep = re.search(ner_words[0] + ' (.*)' + ner_words[1], example)

to_tokenize = tagging_prep.group(1)

# tokenize and tag this phrase
text = word_tokenize(to_tokenize)
pos = nltk.pos_tag(text)


# find verbs and associated subordinating conjunctions (IN)/to (TO)

verb_types = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
relationship = ''
for i in range(len(pos)):
  if pos[i][1] in verb_types:
    relationship = relationship + pos[i][0]
    if pos[i+1][1] == 'IN' or 'TO':
      relationship = relationship + '_' + pos[i+1][0]


# form triple
triple = [ner_words[0], relationship, ner_words[1]]

print(triple)


['Sarah', 'live_in', 'London']


In [41]:
# see what NER outputs for more complex sentences
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

from nltk.tokenize import word_tokenize, sent_tokenize
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

with open('text1-chunk.txt', 'r') as file:
    data = file.read()


tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)

data = sent_tokenize(data)

for i in data:
  ner_results = nlp(i)

  ner_words = []

  # identify "start" word and "end" word
  for j in ner_results:
    word = j.get('entity') + ' ' + j.get('word')
    ner_words.append(word)

  print(i)
  print(ner_words)
  print('\n')

Trafficking Culture - Researching the global traffic in looted objects   The Aidonia Treasure is a collection of Mycenaean gold and jewelry returned to Greece in 1996 and thought to have been robbed from a cemetery at Aidonia in the late 1970s.
['B-LOC Aid', 'I-LOC ##onia', 'I-MISC Treasure', 'B-MISC My', 'I-MISC ##ce', 'I-MISC ##nae', 'I-MISC ##an', 'B-LOC Greece', 'B-LOC Aid', 'I-LOC ##onia']


Aidonia is a village ten kilometres northwest of Nemea in the northeastern Peloponnesian district of Corinthia in Greece.
['B-LOC Aid', 'I-LOC ##onia', 'B-LOC N', 'I-LOC ##em', 'I-LOC ##ea', 'B-MISC P', 'B-MISC ##elo', 'I-MISC ##po', 'I-MISC ##nnes', 'I-MISC ##ian', 'B-LOC Co', 'I-LOC ##rin', 'I-LOC ##thi', 'I-LOC ##a', 'B-LOC Greece']


In November 1976, a Mycenaean (fifteenth-century BC) cemetery was discovered not far from the village, and the tombs were dug out illegally by local archaiokapiloi (a term used for looters in Greece) (Miller 1997: 40; Waxman 2007: 347).
['B-MISC My', 'I-MISC #